In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.utils.data as udata
import torch.optim as optim
import torch.nn.functional as F

# PyTorch geometric
import torch_geometric
import torch_geometric.data as geom_data
import torch_geometric.nn as geom_nn
from torch_geometric.nn import GCNConv

# PL callbacks
#from lightning.pytorch.callbacks import ModelCheckpoint
#from torch import Tensor


#from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

In [93]:
epochs        = 30
n_layers      = 3
n_nodes       = 36
n_outputs     = 100
drop_out_p    = 0.0
learning_rate = 0.001
batch_size    = 100
verbose       = True

#x=60um  y=72um
# tanh
# adam
# pmax>10 && pmax>5

In [94]:
path = './'
#df_train = pd.read_csv('csv_TB3_Run21_croci450um_W3_285V_150kWfm.csv', sep=',')
df_train = pd.read_csv('csv_croci450-RSD2_Run111_285V_49.3pc_W3.csv', sep=',')
#df_test = pd.read_csv('csv_TB3_Run20_croci450um_W3_285V_40kWfm.csv', sep=',')
df_test = pd.read_csv('csv_TB3_Run20_croci450um_W3_285V_40kWfm.csv', sep=',')

df_train = df_train.dropna(axis=1)
df_test = df_test.dropna(axis=1)
df_train = df_train.dropna(axis=0)
df_test = df_test.dropna(axis=0)

#df_train = df_train.astype(double)
#df_test = df_test.astype(double)

Train = df_train
Test = df_test

x_low = -5.475
x_up = -4.575
y_low = -1.232
y_up = -0.332


train = Train[ (Train['x_tracker']>(x_low) ) & (Train['x_tracker']<(x_up)) \
              & (Train['y_tracker']>(y_low)) & (Train['y_tracker']<(y_up)) ]

test = Test[ (Test['x_tracker']>x_low) & (Test['x_tracker']<x_up) & (Test['y_tracker']>y_low) \
              & (Test['y_tracker']<y_up) ]

train['x_tracker'] += -0.05
#test ['x_tracker'] += 5.0
train['y_tracker'] += 0.05
#test ['y_tracker'] += 1.0

train = train[ (train['pmax6']>train['pmax1']) & (train['pmax6']>train['pmax2']) & (train['pmax6']>train['pmax3']) \
        & (train['pmax6']>train['pmax4']) & (train['pmax6']>train['pmax5']) & (train['pmax6']>train['pmax7']) \
        & (train['pmax6']>train['pmax8']) & (train['pmax6']>train['pmax9']) & (train['pmax6']>train['pmax11']) \
        & (train['pmax6']>train['pmax12']) & (train['pmax6']>train['pmax13']) & (train['pmax6']>train['pmax14']) ]

test = test[ (test['pmax6']>test['pmax1']) & (test['pmax6']>test['pmax2']) & (test['pmax6']>test['pmax3']) \
        & (test['pmax6']>test['pmax4']) & (test['pmax6']>test['pmax5']) & (test['pmax6']>test['pmax7']) \
        & (test['pmax6']>test['pmax8']) & (test['pmax6']>test['pmax9']) & (test['pmax6']>test['pmax11']) \
        & (test['pmax6']>test['pmax12']) & (test['pmax6']>test['pmax13']) & (test['pmax6']>test['pmax14']) ]

train = train[ (train['pmax6']>=10) ]
test = test[ (test['pmax6']>=5) ]
train = train[ (train['negpmax6']>=-100) ]
test = test[ (test['negpmax6']>=-100) ]

print("Total training events = ", len(train), len(test))

y_train = train.iloc[:,:2]
x_train = train.drop(['x_tracker','y_tracker','x_rsd','y_rsd','t','t_reco','pmax0','negpmax0','tmax0'],axis=1)
x_train = x_train.drop(['pmax1','pmax2','pmax12','pmax13','negpmax1','negpmax2','negpmax12','negpmax13'],axis=1)
x_train = x_train.iloc[:,:18]
#x_train = x_train.iloc[:,:26]
#x_train['diff3'] = x_train['pmax3']+x_train['negpmax3']


y_test = test.iloc[:,:2]
x_test = test.drop(['x_tracker','y_tracker','x_rsd','y_rsd','t','t_reco','pmax0','negpmax0','tmax0'],axis=1)
x_test = x_test.drop(['pmax1','pmax2','pmax12','pmax13','negpmax1','negpmax2','negpmax12','negpmax13'],axis=1)
x_test = x_test.iloc[:,:18]
#x_test = x_test.iloc[:,:26]
#x_test['diff3'] = x_test['pmax3']+x_test['negpmax3']

z_train = x_train.iloc[:,:9].div(x_train['pmax6'], axis=0)
z_test = x_test.iloc[:,:9].div(x_test['pmax6'], axis=0)

n_var     = len(x_test.columns)
n_outputs = len(y_test.columns)
print(x_test.columns, n_var)
print(y_test.columns, n_outputs)

y_train = torch.tensor(y_train.values)
x_train = torch.tensor(x_train.values)
z_train = torch.tensor(z_train.values)

y_test = torch.tensor(y_test.values)
x_test = torch.tensor(x_test.values)
z_test = torch.tensor(z_test.values)


m_train = x_train.mean(0, keepdim=True)
s_train = x_train.std(0, unbiased=False, keepdim=True)
x_train -= m_train
x_train /= s_train

m_train_y = y_train.mean(0, keepdim=True)
s_train_y = y_train.std(0, unbiased=False, keepdim=True)
y_train -= m_train_y
y_train /= s_train_y

m_test = x_test.mean(0, keepdim=True)
s_test = x_test.std(0, unbiased=False, keepdim=True)
x_test -= m_test
x_test /= s_test

m_test_y = y_test.mean(0, keepdim=True)
s_test_y = y_test.std(0, unbiased=False, keepdim=True)
y_test -= m_test_y
y_test /= s_test_y


#train_tensor = udata.TensorDataset(x_train, y_train) 
#test_tensor  = udata.TensorDataset(x_test,  y_test) 

#loader_train = udata.DataLoader(dataset=train_tensor, batch_size=batch_size, num_workers=0, shuffle=True)
#loader_test  = udata.DataLoader(dataset=test_tensor,  batch_size=batch_size, num_workers=0, shuffle=True)

Total training events =  21957 918
Index(['pmax3', 'pmax4', 'pmax5', 'pmax6', 'pmax7', 'pmax8', 'pmax9', 'pmax11',
       'pmax14', 'negpmax3', 'negpmax4', 'negpmax5', 'negpmax6', 'negpmax7',
       'negpmax8', 'negpmax9', 'negpmax11', 'negpmax14'],
      dtype='object') 18
Index(['x_tracker', 'y_tracker'], dtype='object') 2


/tmp/ipykernel_547/1535561950.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['x_tracker'] += -0.05
/tmp/ipykernel_547/1535561950.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['y_tracker'] += 0.05


In [95]:
train_tensor = udata.TensorDataset(x_train, z_train, y_train)
loader_train = udata.DataLoader(dataset=train_tensor, batch_size=batch_size, num_workers=0, shuffle=True)

test_tensor = udata.TensorDataset(x_test, z_test, y_test)
loader_test = udata.DataLoader(dataset=test_tensor, batch_size=batch_size, num_workers=0, shuffle=True)

In [96]:
# Define GNN model
class GNN(torch.nn.Module):
    def __init__(self, n_var, n_nodes, n_outputs, drop_out_p):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(n_var, n_nodes)
        self.conv2 = GCNConv(n_nodes, n_outputs)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=drop_out_p, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GNN(n_var=n_var, n_nodes=n_nodes, n_outputs=n_outputs, drop_out_p=drop_out_p)

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=1e-3)
#optimizer = optim.SGD(model.parameters(), lr = learning_rate, weight_decay=1e-3)

scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.95, last_epoch=-1, verbose=verbose)

Adjusting learning rate of group 0 to 1.0000e-03.


In [97]:
# training and validation
training_losses_total = np.zeros(epochs)
validation_losses_total = np.zeros(epochs)
for epoch in range(epochs):
    if verbose: print("Beginning epoch " + str(epoch))
    # training
    train_loss_total = 0
    for i, data in enumerate(loader_train,0):
        x, z, y = data
        print(y.size())
        #print(x)
        #print(z)
        #print(y)
        model.double()
        model.train()
        model.zero_grad()
        optimizer.zero_grad()
        output = model(x, z.long())
        batch_loss_total = criterion(output, y)        
        batch_loss_total.backward()
        optimizer.step()
        model.eval()
        train_loss_total += batch_loss_total.item()
    train_loss_total /= len(loader_train)
    training_losses_total[epoch] = train_loss_total
    if np.isnan(train_loss_total):
        print("nan in training")
        break
    if verbose: print("t_total: "+ str(train_loss_total))

    # validation
    val_loss_total = 0
    for i, data in enumerate(loader_test):
        x, z, y = data
        output = model(x, z.long())
        batch_loss_total = criterion(output, y)        
        val_loss_total += batch_loss_total.item()
    val_loss_total /= len(loader_test)
    #scheduler.step()
    validation_losses_total[epoch] = val_loss_total
    if np.isnan(val_loss_total):
        print("nan in val")
        break
    if verbose: print("v_total: "+ str(val_loss_total))
    
    # save the model
    model.eval()
    #modelLocation = "{}/net_{}.pth".format(".",epoch)
    #torch.save(model.state_dict(), modelLocation)
    
print("Finished")
print("Final loss train:", training_losses_total[-1])
print("Final loss test:",  validation_losses_total[-1])

Beginning epoch 0
torch.Size([100, 2])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 100 but got size 2 for tensor number 1 in the list.

In [ ]:
# plot loss/epoch for training and validation sets
print("Making basic validation plots")
training_total = plt.plot(training_losses_total, label='training_total')
validation_total = plt.plot(validation_losses_total, label='validation_total')
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()
#plt.savefig("loss_plot.png")